In [1]:
import pandas as pd
import numpy as np
import math
#ONLY TO TEST ACCUARACY METHODS: FROM HERE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
#TO HERE
from numpy import random
import matplotlib.pyplot as plt

In [2]:
breastCancer = pd.read_csv('Data/breast-cancer-wisconsin.csv', header=None)
glass = pd.read_csv('Data/glass.csv', header=None)
houseVotes = pd.read_csv('Data/house-votes-84.csv', header=None)
iris = pd.read_csv('Data/iris.csv', header=None)
soyBean = pd.read_csv('Data/soybean-small.csv', header=None)
datasets = [breastCancer, glass, houseVotes, iris, soyBean]
breastCancer

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


In [3]:
#Removes instances with missing values
def removesMissingValues(df):
    checkItems = ["?"]
    missingRows = []
    for i, row in df.iterrows():
        isMissing = False
        for j, value in row.items():
            for k in checkItems:
                if value == k or value == math.nan:
                    isMissing = True
        if isMissing:
            missingRows.append(i)
    df.drop(missingRows, axis=0, inplace=True)
    return df

In [4]:
def fillMean(df):
    checkItems = ["?"]
    for col in df:
        missingIndex = []
        for index, value in df[col].items():
            for k in checkItems:
                if value == k or value == math.nan:
                    isMissing = True
        if len(missingIndex) > 0:
            newCol = df[col]
            newCol.drop(missingIndex)
            mean = newCol.mean()
            for i in missingIndex:
                df[col][i] = mean
    return df

In [5]:
def fillForward(df):
    checkItems = ["?"]
    for col in df:
        for index, value in df[col].items():
            for k in checkItems:
                if value == k or value == math.nan:
                    if index+1 >= len(df[col]):
                        df[col][index] = df[col][0]
                    else:
                        df[col][index] = df[col][index+1]
    return df

In [6]:
def fillBackward(df):
    checkItems = ["?"]
    for col in df:
        for index, value in df[col].items():
            for k in checkItems:
                if value == k or value == math.nan:
                    if index-1 < 0:
                        df[col][index] = df[col][len(df[col]) - 1]
                    else:
                        df[col][index] = df[col][index-1]
    return df

In [7]:
def labelEncode(df):
    for col in df:
        if type(df[col][0]) == str:
            labels = []
            for index, value in df[col].items():
                isDup = False
                for i in labels:
                    if i == value:
                        isDup = True
                if not isDup:
                    labels.append(value)
            for index, value in df[col].items():
                for i in labels:
                    if i == value:
                        df[col][index] = labels.index(value)
    return df

In [8]:
def oneHotEncode(df):
    for col in df:
        if type(df[col][0]) == str:
            labels = []
            for index, value in df[col].items():
                isDup = False
                for i in labels:
                    if i == value:
                        isDup = True
                if not isDup:
                    labels.append(value)
            for i in labels:
                temp = np.zeros(df[col].size)
                for index, value in df[col].items():
                        if i == value:
                            temp[index] = 1
                df.insert(col, i, temp)
            df.drop(df.columns[[col+len(labels)]], axis=1, inplace=True)
    return df

In [9]:
def getLabels(df):
    labels = []
    for index, value in df.items():
        isDup = False
        for i in labels:
            if i == value:
                isDup = True
        if not isDup:
            labels.append(value)
    return labels

In [10]:
#FOR TESTING PURPOSES ONLY
X = iris.iloc[:,0:3].values
Y = iris.iloc[:, 4]
X_train, X_test, y_train, y_test =train_test_split(X,Y,test_size= 0.25, random_state=0)
sc_X = StandardScaler() 
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)
classifer1 = GaussianNB()
classifer1.fit(X_train, y_train)
y_pred1 = classifer1.predict(X_test)
print(classification_report(y_test, y_pred1))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        13
Iris-versicolor       1.00      0.56      0.72        16
 Iris-virginica       0.56      1.00      0.72         9

       accuracy                           0.82        38
      macro avg       0.85      0.85      0.81        38
   weighted avg       0.90      0.82      0.82        38



In [11]:
def confusionMatrix(pred, truth, label):
    matrix = []
    for i in range(len(label)):
        matrix.append([0] * len(label))
    for i in range(len(pred)):
        matrix[label.index(pred[i])][label.index(truth[i])] += 1
    return matrix
print(getLabels(iris.iloc[:,4]))
label = getLabels(iris.iloc[:,4])
confusionMatrix(np.array(y_pred1), np.array(y_test), label)

['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']


[[13, 0, 0], [0, 9, 0], [0, 7, 9]]

In [12]:
def truePositive(conmat):
    TP = []
    for i in range(len(conmat)):
        TP.append(conmat[i][i])
    return TP
truePositive(confusionMatrix(np.array(y_pred1), np.array(y_test), label))

[13, 9, 9]

In [13]:
def trueNegative(conmat):
    TN = []
    for i in range(len(conmat)):
        count = 0
        for j in range(len(conmat)):
            for k in range(len(conmat)):
                if j != i and k != i:
                    count += conmat[j][k]
        TN.append(count)
    return TN
trueNegative(confusionMatrix(np.array(y_pred1), np.array(y_test), label))

[25, 22, 22]

In [14]:
def falsePositive(conmat):
    FP = []
    for i in range(len(conmat)):
        count = 0
        for j in range(len(conmat)):
            if i != j:
                count += conmat[i][j]
        FP.append(count)
    return FP
falsePositive(confusionMatrix(np.array(y_pred1), np.array(y_test), label))

[0, 0, 7]

In [15]:
def falseNegative(conmat):
    FN = []
    for i in range(len(conmat)):
        count = 0
        for j in range(len(conmat)):
            if i != j:
                count += conmat[j][i]
        FN.append(count)
    return FN
falseNegative(confusionMatrix(np.array(y_pred1), np.array(y_test), label))

[0, 7, 0]

In [16]:
def precision(pred, truth, label):
    conmat = confusionMatrix(np.array(y_pred1), np.array(y_test), label)
    TP = truePositive(conmat)
    FP = falsePositive(conmat)
    prec = []
    for i in range(len(conmat)):
        prec.append((TP[i])/(TP[i] + FP[i]))
    return prec
precision(y_pred1, y_test, label)

[1.0, 1.0, 0.5625]

In [17]:
def recall(pred, truth, label):
    conmat = confusionMatrix(np.array(y_pred1), np.array(y_test), label)
    TP = truePositive(conmat)
    FN = falseNegative(conmat)
    prec = []
    for i in range(len(conmat)):
        prec.append((TP[i])/(TP[i] + FN[i]))
    return prec
recall(y_pred1, y_test, label)

[1.0, 0.5625, 1.0]

In [18]:
def fScore(b, pred, truth, label):
    prec = precision(pred, truth, label)
    rec = recall(pred, truth, label)
    f = []
    for i in range(len(label)):
        Fb = (1 + pow(b, 2))*(prec[i]*rec[i])/((pow(b,2) * prec[i]) + rec[i])
        f.append(Fb)
    return f

def f1Score(pred, truth, label):
    return fScore(1, pred, truth, label)

f1Score(y_pred1, y_test, label)

[1.0, 0.72, 0.72]

In [19]:
def shuffle(df):
    randoms = int(df.shape[0] * 0.1)
    indices = []
    rows = []
    
    for i in range(randoms):
        r = random.randint(df.shape[0])
        indices.append(r)
        rows.append(np.array(df.iloc[[r]]))
        
    df.drop(indices, axis=0, inplace=True)
    df = df.reset_index(drop=True)
    
    rest = []
    for i in range(df.shape[0]):
        rest.append(np.array(df.iloc[[i]]))
        
    for i in rows:
        r = random.randint(len(rest))
        rest.insert(r, i)
    for i in range(len(rest)):
        rest[i] = rest[i][0]
        
    df = pd.DataFrame(rest)
    return df

shuffle(iris)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,5.9,3.0,5.1,1.8,Iris-virginica
146,6.7,3.3,5.7,2.5,Iris-virginica
147,6.7,3.0,5.2,2.3,Iris-virginica
148,6.3,2.5,5.0,1.9,Iris-virginica


In [ ]:
def plot(lossFunc1, lossFunc2, results):
    for i, result in enumerate(results):
        